In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Medha Trust\\Desktop\\Auto\\Chest_Cancer_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Medha Trust\\Desktop\\Auto\\Chest_Cancer_Classification'

In [5]:
# 4. Update the entity 
from dataclasses import dataclass
from pathlib import Path


# Same variables also written in config.yaml
# This is a data return types.
# It will return the configuration of the data
# We can access the class variable without any type of self is called dataclass.    
@dataclass(frozen=True)     # Dataclass decorator
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# 5. Update the configuration manager in src config
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # Here the artifacts_root folder is created.

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
# Code for downloading data from google drive using config and gdown
# 6. Update the components

class DataIngestion:
    def __init__(self,config):
        self.config = config

    def download_file(self):
        """ Fetch data from URL.
        """

        try:
            dataset_url=self.config.source_url
            zip_download_dir=self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix='https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        """"
            zip_filepath : str
            Extracts the zip file into the data directory
            Function returns None
        """

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
# 7. Update the pipeline
# We can create the pipeline using zenml

try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-27 17:25:51,416: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-27 17:25:51,418: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-27 17:25:51,419: INFO: common: Created directory at : artifacts]
[2024-05-27 17:25:51,420: INFO: common: Created directory at : artifacts/data_ingestion]
[2024-05-27 17:25:51,421: INFO: 2808208007: Downloading data from https://drive.google.com/file/d/1XIAroODvoYEPNA8UwI38x7ZXsOneWRpj/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1XIAroODvoYEPNA8UwI38x7ZXsOneWRpj
To: c:\Users\Medha Trust\Desktop\Auto\Chest_Cancer_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 7.50M/7.50M [00:02<00:00, 3.75MB/s]

[2024-05-27 17:25:55,969: INFO: 2808208007: Downloaded data from https://drive.google.com/file/d/1XIAroODvoYEPNA8UwI38x7ZXsOneWRpj/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
